# Linear programming
### Забара Илья

## №1

Выпишем задачу ЛП.

Максимизируемая функкция будет такая: $5y_1+7y_2+13y_3\longrightarrow \displaystyle{\max_x}$.

Ограничение на производственную мощность, с учётом, что завод не может все 6 дней работать над одним и тем же: $\dfrac{y_1}{5000}+\dfrac{y_2}{4000}+\dfrac{y_3}{2000}\leq6$.

Ограничение на размер склада: $\dfrac{30y_1}{1000}+\dfrac{50y_2}{1000}+\dfrac{220y_3}{1000}\leq 1500$.

Ограничения коммерческого соглашения: $y_1\geq 4500, \ y_3 \geq 4000$.

Ограничение спроса: $y_1\leq10000, \ y_2\leq14000, \ y_3\leq7000$.

Все переменные должны быть неотрицательными: $y_1, y_2, y_3 \geq0$.

Импортируем модуль среды из pyomo и создадим модель:


In [112]:
from pyomo.environ import *

model1 = ConcreteModel()

Теперь определим наши переменные, уже с учётом неотрицательности. Параметр within используется в этих объявлениях параметров для определения ожидаемых свойств параметров (неотрицательности). Компонент Var используется для определения переменных:

In [113]:
model1.y1 = Var(within=NonNegativeReals)
model1.y2 = Var(within=NonNegativeReals)
model1.y3 = Var(within=NonNegativeReals)

Теперь нам надо определить ограничения:

In [114]:
model1.prod = Constraint(expr = model1.y1/5000 + model1.y2/4000 + model1.y3/2000 <= 6)
model1.storage = Constraint(expr = 30*model1.y1/1000 + 50*model1.y2/1000 + 220*model1.y3/1000 <= 1500)
model1.min_headphone = Constraint(expr = -model1.y1 <= -4500)
model1.min_laptop = Constraint(expr = -model1.y3 <= -4000)
model1.max_headphone = Constraint(expr = model1.y1 <= 10000)
model1.max_phone = Constraint(expr = model1.y2 <= 14000)
model1.max_laptop = Constraint(expr = model1.y3 <= 7000)

Далее задаём *максимизируемую* целевую функцию (за максимизацию отвечает аргумент sense). 

In [115]:
model1.profit = Objective(expr = 5*model1.y1 + 7*model1.y2 + 12*model1.y3, sense=maximize)

Теперь решаем задачу по построенной модели солвером GLPK и выводим результат на экран:

In [116]:
solver1 = SolverFactory('glpk')
solver1.solve(model1)

print("Оптимальные значения y1, y2 и y3: ", model1.y1(), model1.y2(), model1.y3())
print("Максимальная прибыль: ", model1.profit())

Оптимальные значения y1, y2 и y3:  10000.0 6400.0 4000.0
Максимальная прибыль:  142800.0


Теперь проведём анализ чувствительности ограничений в данной задаче. Для этого решим двойственную задачу и посмотрим на получившиеся при этом двойственные переменные.

Для нахождения решения двойственной задачи воспользуемся также библиотекой pyomo, взяв за основу решение исходной задачи. Можно сразу получать значения двойственных переменных. Об можно узнать из документации. Ссылка на документацию с примерами, взятыми за основу: https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Suffixes.html .

In [117]:
solver = SolverFactory('glpk')
model1.dual = Suffix(direction=Suffix.IMPORT)  # создаём суффикс объекта для хранения двойственых переменных
solver.solve(model1)  # ещё раз прогоняем решение

# выводим двойственные значения
print("Оптимальное решение двойственной задачи для каждого из ограничений")
for c in model1.component_objects(Constraint, active=True):
    for index in c:
        print(c, ": lambda_i = ",  model1.dual[c[index]])

Оптимальное решение двойственной задачи для каждого из ограничений
prod : lambda_i =  0.0
storage : lambda_i =  140.0
min_headphone : lambda_i =  0.0
min_laptop : lambda_i =  18.8
max_headphone : lambda_i =  0.8
max_phone : lambda_i =  0.0
max_laptop : lambda_i =  0.0


Из физического смысла множителей Лагранжа можно сделать вывод о том, какое ограничение можно было бы ослабить, чтобы максимально увеличить прибыль. Наибольший множитель Лагранжа соответствует ограничению storage, задающему максимальный размер места хранения товаров.
Это означает, что оптиимальное решение наболее чувствительно именно к этому ограничению. Остальные ограничения либо не влияют, либо существенно меньше влияют на получаемое решение. Это известно из теории  возмущённой задачи (рассмотрено на одном из предыдущих занятий).

Давайте теперь создадим модель model2, в которой "расширим" место хранения товаров. Зададим максимальный объём не 1500, а уже, например, 2500:

In [118]:
model2 = ConcreteModel()

model2.y1 = Var(within=NonNegativeReals)
model2.y2 = Var(within=NonNegativeReals)
model2.y3 = Var(within=NonNegativeReals)

model2.prod1 = Constraint(expr = model2.y1<= 5000*6)
model2.prod2 = Constraint(expr = model2.y2<= 4000*6)
model2.prod3 = Constraint(expr = model2.y3<= 2000*6)
model2.storage = Constraint(expr = 30*model2.y1/1000 + 50*model2.y2/1000 + 220*model2.y3/1000 <= 2500)  # расширили объём места хранения
model2.min_headphone = Constraint(expr = -model2.y1 <= -4500)
model2.min_laptop = Constraint(expr = -model2.y3 <= -4000)
model2.max_headphone = Constraint(expr = model2.y1 <= 10000)
model2.max_phone = Constraint(expr = model2.y2 <= 14000)
model2.max_laptop = Constraint(expr = model2.y3 <= 7000)

model2.profit = Objective(expr = 5*model2.y1 + 7*model2.y2 + 12*model2.y3, sense=maximize)

solver = SolverFactory('glpk')
model2.dual = Suffix(direction=Suffix.IMPORT)
solver.solve(model2)

print("Values of y1, y2 and y3: ", model2.y1(), model2.y2(), model2.y3())
print("Максимальная прибыль с расшренным складом: ", model2.profit())

Values of y1, y2 and y3:  10000.0 14000.0 6818.18181818182
Максимальная прибыль с расшренным складом:  229818.18181818182


Видим, что действительно прибыль значительно выросла, что говорит о том, что ограничение на размер хранилища товаров действительно сильно влияет на макисмальную прибыль.

На всякий случай убедимся, что, например, ограничение min_laptops действительно меньше влияет на максимальную прибыль. Для этого создадим и решим model3, в которой разрешим производить количество чехлов для ноутбуков не от 4000 единиц, а от 3000 единиц, дав тем самым больше свободы для данной модели. Возмущение при этом такое же, как в model2 (увеличили правую часть этого ограничения (умноженного на -1 для классической постановки) на 1000 также как и в model2):

In [119]:
model3 = ConcreteModel()

model3.y1 = Var(within=NonNegativeReals)
model3.y2 = Var(within=NonNegativeReals)
model3.y3 = Var(within=NonNegativeReals)

model3.prod1 = Constraint(expr = model3.y1<= 5000*6)
model3.prod2 = Constraint(expr = model3.y2<= 4000*6)
model3.prod3 = Constraint(expr = model3.y3<= 2000*6)
model3.storage = Constraint(expr = 30*model3.y1/1000 + 50*model3.y2/1000 + 220*model3.y3/1000 <= 1500)  
model3.min_headphone = Constraint(expr = -model3.y1 <= -4500)
model3.min_laptop = Constraint(expr = -model3.y3 <= -3000)  # ослабили ограничение на min_laptops
model3.max_headphone = Constraint(expr = model3.y1 <= 10000)
model3.max_phone = Constraint(expr = model3.y2 <= 14000)
model3.max_laptop = Constraint(expr = model3.y3 <= 7000)

model3.profit = Objective(expr = 5*model3.y1 + 7*model3.y2 + 12*model3.y3, sense=maximize)

solver = SolverFactory('glpk')
model3.dual = Suffix(direction=Suffix.IMPORT)
solver.solve(model3)

print("Values of y1, y2 and y3: ", model3.y1(), model3.y2(), model3.y3())
print("Максимальная прибыль с ослабленным ограничением min_laptops: ", model3.profit())

Values of y1, y2 and y3:  10000.0 10800.0 3000.0
Максимальная прибыль с ослабленным ограничением min_laptops:  161600.0


Видим, что прибыль выросла, но не так сильно, как после изменения ограничения storage на размер хранилища. Таким образом показали, что именно ограничение storage на размер хранилища наиболее сильно влияет на максимальную прибыль

**<u>Ответ:</u>** задача поставлена, решена с помощью pyomo, а также сделаны выводы о наболее влиющем на максимальную прибыль ограничений с помощью анализа чувствительности: ослабление ограничения на размер места хранения товаров влияет в наибольшей степени.

## №2

$x^*=(7/3, 0, 1/3)^T - $ проверить на оптимальность

Сразу умножим целевую функцию на $-1$, чтобы воспользоваться достаточным условием оптимальности вершины симплекса в формулировке, данной на занятии. Также умножим на $-1$ условие неотрицательности, чтобы в дальнейшем удобно выписать матрицу ограничений $A$. Решаемая задача такая:

$$
\begin{cases} -9x_1-3x_2-7x_3\longrightarrow \displaystyle{\min_x} \\ 
2x_1+x_2+3x_3\leq6 \\
5x_1+4x_2+x_3\leq12 \\
3x_3\leq1 \\
-x_1, -x_2, -x_3 \leq 0 \end{cases}
$$

Заметим, что $x^*$ есть вершина симплекса (одна из), т.к. ограничения $\begin{cases} 5x_1+4x_2+x_3\leq12 \\ 3x_3\leq1 \\ -x_2\leq 0 \end{cases}$ в этой точке выполнены как равенства.

Все ограничения: $Ax\leq b$, где $A = \begin{pmatrix}
2 & 1 & 3\\
5 & 4 & 1\\
0 & 0 & 3\\
-1 & 0 & 0\\
0 & -1 & 0\\
0 & 0 & -1
\end{pmatrix}$ и $b = \begin{pmatrix} 6 \\ 12 \\ 1 \\ 0 \\ 0 \\ 0  \end{pmatrix}$.

Базис $B$, задающий угловую точку $x^*$, получаетс такой: $B = \{ 2, 3, 5 \}$. Он является допустимым, так как угловая точка $x^*$ удовлетворяет всем остальным ограничениям.

Подматрица и подвектор, соответствующие базису $B$: $A_B = \begin{pmatrix} 5 & 4 & 1 \\ 0 & 0 & 3 \\ 0 & -1 & 0 \end{pmatrix} \Rightarrow A_B^{-1} = \dfrac{1}{15} \begin{pmatrix} 3 & -1 & 12 \\ 0 & 0 & -15 \\ 0 & 5 & 0 \end{pmatrix}$, $b_B = \begin{pmatrix} 12 \\ 1 \\ 0 \end{pmatrix}$.

Коэффициенты разложения $\lambda_B=A_B^{-T}\cdot c$, где $c = \begin{pmatrix} -9 \\ -3 \\ -7 \end{pmatrix} - $ вектор градиента целевой *минимизируемой* функции.

После пермножения имеем: $\lambda_B = \dfrac{1}{15} \begin{pmatrix} -27 \\ -16 \\ -63 \end{pmatrix}$.

$\lambda_B \preceq 0 \Rightarrow x^* = (7/3, 0, 1/3)^T - $ решение данной задачи линейного программирования.

**<u>Ответ:</u>** доказали оптимальность точки.

## №3

$\begin{cases} x_1-x_2 \longrightarrow \displaystyle{\min_x} \\ 2x_1+x_2\geq 3 \\ 3x_1-x_2\leq 7 \\ x_1\geq 0 \ \end{cases}$ нужно преобразовать к стандартному виду.

Заметим, что нет условия неотрицательности на $x_2\Rightarrow$ введём две неотрицательные перменные, композиция которых будет задавать переменную $x_2$, при этом никак не ограничивая её знак: $x_2 = x_2^+ - x_2^-$, где $x_2^+, x_2^- \geq 0$. Тогда:

$$\begin{cases} x_1-x_2^+ + x_2^- \longrightarrow \displaystyle{\min_x} \\ 2x_1+x_2^+-x_2^-\geq 3 \\ 3x_1-x_2^+ +x_2-\leq 7 \\ x_1, x_2^+, x_2^-\geq 0 \ \end{cases}$$

Осталось преобразовать неравенства к равенствам. В первом стоит знак $\leq$, значит от левой части этого ограничения надо отнять что-то положительное (новая перменная $x_3$). Во втором ограничении наоборот, прибавить что-то положительное. Тогда:

$\begin{cases} 2x_1+x_2^+-x_2^--x_3=3 \\ 3x_1-x_2^++x2^-+x_4=7 \end{cases} \Leftrightarrow \begin{cases} 2x_1+x_2^+-x_2^--x_3-0\cdot x_4=3 \\ 3x_1-x_2^++x2^-+0\cdot x_3+x_4=7 \end{cases} \Leftrightarrow Ax = b$.

Тут $A = \begin{pmatrix} 2 & 1 & -1 & -1 & 0 \\ 3 & -1 & 1 & 0 & 1 \end{pmatrix}, b = \begin{pmatrix} 3 \\ 7 \end{pmatrix}, x = (x_1, x_2^+, x_2^-, x_3, x_4)^T \in \mathbb{R}^5$.

$x_1, x_2^+, x_2^-, x_3, x_4 \geq 0 \Leftrightarrow x \geq 0$.

Целевая функция: $x_1 - x_2^+ + x_2^- + 0\cdot x_3 + 0\cdot x_3 = c^Tx \Leftrightarrow c = (1, -1, 1, 0, 0)^T \in \mathbb{R}^5$.

Итого пришли к задаче:

$$
\begin{cases} c^Tx \longrightarrow \displaystyle{\min_{x\in\mathbb{R}^5}} \\ Ax=b \\ x\geq 0 \end{cases},
$$

где $ c = (1, -1, 1, 0, 0)^T, \ x = (x_1, x_2^+, x_2^-, x_3, x_4)^T, \ A = \begin{pmatrix} 2 & 1 & -1 & -1 & 0 \\ 3 & -1 & 1 & 0 & 1 \end{pmatrix}, b = \begin{pmatrix} 3 \\ 7 \end{pmatrix}$.


**<u>Ответ:</u>** перешли к каноническому виду.

## №4

Домножим макисимзируемую фуннкцию на $-1$, чтобы она  стала минимизируемой. Задача ЛП теперь такая:

$\begin{cases} -4x_1-5x_2-5x_3 \longrightarrow \displaystyle{\min_{x\in\mathbb{R}^3}} \\ 2x_1-x_2+2x_3\leq 9 \\ 3x_1+5x_2+4x_3\leq8 \\ x_1+x_2+2x_3\leq2 \\ x_1, x_2, x_3 \geq 0 \end{cases}$

Нужно найти оптимальное решение симплекс-методом, а также решить двойственную задачу.

Задачу разрешено решать програмно на Python. Воспользуемся, также как и в №1 библиотекой pyomo. Солвер GLPK по умолчанию использует симплекс-метод. В следующей ячейке приведём всю программу сразу:

In [120]:
from pyomo.environ import *

model = ConcreteModel()  # создали модель

# задаём переменные
model.x1 = Var(within=NonNegativeReals)
model.x2 = Var(within=NonNegativeReals)
model.x3 = Var(within=NonNegativeReals)

# задаём целевой функционал
model.obj = Objective(expr = 4*model.x1 + 5*model.x2 + 4*model.x3, sense=maximize)

# задаём ограничения
model.con1 = Constraint(expr = 2*model.x1 - model.x2 + 2*model.x3 <= 9)
model.con2 = Constraint(expr = 3*model.x1 + 5*model.x2 + 4*model.x3 <= 8)
model.con3 = Constraint(expr = model.x1 + model.x2 + 2*model.x3 <= 2)

# решение модели солвером GLPK
SolverFactory('glpk').solve(model)

# результат
print(f"x_opt = ({model.x1.value}, {model.x2.value}, {model.x3.value})")
print("Оптимальное значение -f(x):", model.obj())

x_opt = (1.0, 1.0, 0.0)
Оптимальное значение -f(x): 9.0


Программа выдала ответ $x^* = (1, 1, 0)^T$ для оптимизационной переменной. Для функции $-f(x)$ оптимальное значение равно $-9$, тогда оптимальное значение в исходное задаче (для максимизации): $f(x)=9$. Проверим на всякий случай на оптимальность это решение, но уже вручную, закрепив тем самым метод проверки оптимальности из №2.

Выданная оптимальная точка является угловой для 2, 3 и 6 ограничений. Подматрица $A_B$ в этом случае $A_B = \begin{pmatrix} 3 & 5 & 4 \\ 1 & 1 & 2 \\ 0 & 0 & -1 \end{pmatrix}$. Обратная к ней матрица $A_B^{-T} = \dfrac{1}{2} \begin{pmatrix} -1 & 1 & 0 \\ 5 & -3 & 0 \\ 6 & -2 & -2 \end{pmatrix}$. Вектор градиента в задаче минимизации $c = \begin{pmatrix} -4 \\ -5 \\ -5 \end{pmatrix}$.

Тогда коэффициенты разложения: $\lambda_B = A_B^{-T}\cdot c = \begin{pmatrix} -\frac{1}{2} \\ -\frac{5}{2} \\ -2 \end{pmatrix} \preceq 0 \Rightarrow$ это действительно оптимальная точка.

Для решения двойственной задачи воспользуемся также библиотекой pyomo, взяв за основу решение исходной задачи. Можно сразу получать значения двойственных переменных. Об можно узнать из документации. Ссылка на документацию с примерами, взятыми за основу: https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Suffixes.html 

In [121]:
solver = SolverFactory('glpk')
model.dual = Suffix(direction=Suffix.IMPORT)  # создаём суффикс объекта для хранения двойственых переменных
solver.solve(model)  # ещё раз прогоняем решения

print("Двойственные значения для каждого из ограничений:")
for c in model.component_objects(Constraint, active=True):
    for index in c:
        print(c, ":",  model.dual[c[index]])

Двойственные значения для каждого из ограничений:
con1 : 0.0
con2 : 0.5
con3 : 2.5


Таким образом мы получили решение двойственной задачи: $\ \nu^* = (0, -1/2, -5/2)^T$. Множители Лагранжа, соответствующие ограничениям неотрицательности переменных здесь не приведены,но нужно помнить о том, что размерность двойственных переменных тут всё-таки равна 6 (просто решение не чувствительно к множителям Лагранжа, соответствующим ограничениям неотрицательности переменных). Почему это именно так, будет понятно дальше.

Для того, чтобы понять есть сильная двойственность или нет, нужно явно выписивать двойственную функцию и подставить в неё значение $\nu^*$.

Лагранжиан исходной задачи: $L = -4x_1-5x_2-5x_3+\nu_1(2x_1-x_2+2x_3-9)+\nu_2(3x_1+5x_2+4x_3-8)+\nu_3(x_1+x_2+2x_3-2)+\nu_4(-x_1)+\nu_5(-x_2)+\nu_6(-x_3)$. 

Линейная по $x_1, x_2, x_3$ часть Лагранжиана должна быть приравнена к нулю, ибо у Лагранжиана тогда при этом не будет коненчного минимума, что приведёт к неопределённости двойственной функции.

Убрав все такие слагаемые получаем двойственную функцию, которая, обратите внимание, завсит только от первых трёх множителей Лагранжа (последние 3 входили в состав линейной по исходным переменным части Лагранжиана): $\boxed{g(\nu)=g(\nu_1, \nu_2, \nu_3)=\displaystyle{\min_{x_1, x_2, x_3}L}=-9\nu_1-8\nu_2-2\nu_3}$. 

Независимость этой функции от множителей Лагранжа, соответствующим ограничениям неотрицательности исходных переменных в исходной задаче, и объясняет то, почему эти ограничения не учавствуют в чувстивтельности оптимального решения.

Теперь, собвственнно, проверим двойственность, подставив полченное солвером решение $\nu^* = (0, -1/2, -5/2)^T$ в функцию $g$: 

$$g(\nu^*)=-9\cdot0+8\cdot\frac{1}{2}+2\cdot\frac{5}{2}=9=d^*$$

При решении исходной задачи мы получили $f(x^*)=9=p^*$. Следовательно, $p^*=d^*=9\Rightarrow$ имеет место сильная двойственность.

**<u>Ответ:</u>** $x^* = (1, 1, 0)^T-$ решение исходной задачи, $\ \nu^* = (0, -1/2, -5/2)^T-$ решение двойственной задачи; есть сильная двойственность.